In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
import scipy as sc

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
Passenger_Id = test['PassengerId'].values
for_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
train = train.drop(for_drop, axis=1)
test = test.drop(for_drop, axis=1)
all_data = pd.concat([train, test], axis=0)

In [8]:
print all_data

      Age Embarked      Fare  Parch  Pclass     Sex  SibSp  Survived
0    22.0        S    7.2500      0       3    male      1       0.0
1    38.0        C   71.2833      0       1  female      1       1.0
2    26.0        S    7.9250      0       3  female      0       1.0
3    35.0        S   53.1000      0       1  female      1       1.0
4    35.0        S    8.0500      0       3    male      0       0.0
5     NaN        Q    8.4583      0       3    male      0       0.0
6    54.0        S   51.8625      0       1    male      0       0.0
7     2.0        S   21.0750      1       3    male      3       0.0
8    27.0        S   11.1333      2       3  female      0       1.0
9    14.0        C   30.0708      0       2  female      1       1.0
10    4.0        S   16.7000      1       3  female      1       1.0
11   58.0        S   26.5500      0       1  female      0       1.0
12   20.0        S    8.0500      0       3    male      0       0.0
13   39.0        S   31.2750      

In [9]:
to_factorize = [label for label in ['Sex', 'Ticket', 'Cabin', 'Embarked'] if not label in for_drop]
factorized_features = [pd.factorize(all_data[label].values, na_sentinel=0)[0] for label in to_factorize]
for i in range(len(to_factorize)):
    all_data[to_factorize[i]] = factorized_features[i]

imp = Imputer()
all_data['Age'] = imp.fit_transform(all_data['Age'].values.reshape(-1, 1))

ohe = OneHotEncoder(sparse=False)
dummy_features = ohe.fit_transform(all_data['Embarked'].values.reshape(-1, 1))
dummy_features = pd.DataFrame(dummy_features, columns=['Embarked = ' + str(x) for x in range(dummy_features.shape[1])])
all_data.index = dummy_features.index
all_data = pd.concat([all_data.drop(['Embarked'], axis=1), dummy_features], axis=1)

In [10]:
print all_data

            Age      Fare  Parch  Pclass  Sex  SibSp  Survived  Embarked = 0  \
0     22.000000    7.2500      0       3    0      1       0.0           1.0   
1     38.000000   71.2833      0       1    1      1       1.0           0.0   
2     26.000000    7.9250      0       3    1      0       1.0           1.0   
3     35.000000   53.1000      0       1    1      1       1.0           1.0   
4     35.000000    8.0500      0       3    0      0       0.0           1.0   
5     29.881138    8.4583      0       3    0      0       0.0           0.0   
6     54.000000   51.8625      0       1    0      0       0.0           1.0   
7      2.000000   21.0750      1       3    0      3       0.0           1.0   
8     27.000000   11.1333      2       3    1      0       1.0           1.0   
9     14.000000   30.0708      0       2    1      1       1.0           0.0   
10     4.000000   16.7000      1       3    1      1       1.0           1.0   
11    58.000000   26.5500      0       1

In [11]:
train, test = all_data[:len(train['Sex'])], all_data[len(train['Sex']):]
test = test.drop(['Survived'], axis=1)

In [12]:
X = train.drop(['Survived'], axis=1).as_matrix()
y = train['Survived'].values
X_test = test.as_matrix()
imp = Imputer(axis=1)
X_test = imp.fit_transform(X_test)

In [13]:
#clf = SVC()
def test_m(model):
    model.fit(X, y)
    ret = cross_val_score(model, X, y, cv=3)
    print ret, np.mean(ret)
    return ret, np.mean(ret)

In [19]:
test_m(LogisticRegression())
scores = []
for reg in np.arange(0, 1, 0.01)[1:]:
    print reg
    ret1, ret2 = test_m(LogisticRegression(C=reg))
    scores.append(ret2)
    
print '\n========', np.arange(0, 1, 0.01)[1:][np.argmax(scores)]

[ 0.78451178  0.78787879  0.78787879] 0.786756453423
0.01
[ 0.6969697   0.72053872  0.75084175] 0.72278338945
0.02
[ 0.71043771  0.75420875  0.78787879] 0.750841750842
0.03
[ 0.73737374  0.76767677  0.8047138 ] 0.769921436588
0.04
[ 0.74074074  0.78451178  0.80808081] 0.777777777778
0.05
[ 0.75420875  0.79461279  0.8047138 ] 0.784511784512
0.06
[ 0.76094276  0.7979798   0.8013468 ] 0.786756453423
0.07
[ 0.76767677  0.7979798   0.7979798 ] 0.787878787879
0.08
[ 0.77104377  0.7979798   0.78787879] 0.785634118967
0.09
[ 0.77441077  0.79461279  0.79124579] 0.786756453423
0.1
[ 0.77441077  0.78451178  0.79124579] 0.783389450056
0.11
[ 0.77104377  0.79124579  0.79124579] 0.784511784512
0.12
[ 0.77104377  0.79124579  0.79124579] 0.784511784512
0.13
[ 0.79124579  0.79461279  0.79124579] 0.792368125701
0.14
[ 0.79124579  0.79461279  0.79124579] 0.792368125701
0.15
[ 0.79124579  0.79124579  0.79124579] 0.791245791246
0.16
[ 0.79124579  0.79461279  0.79124579] 0.792368125701
0.17
[ 0.78787879  0.

In [20]:
scores = []
for weight in np.arange(0, 1, 0.01)[1:]:
    print weight
    ret1, ret2 = test_m(LogisticRegression(C=0.23, class_weight={0.:1, 1.:weight}))
    scores.append(ret2)
print np.arange(0, 1, 0.01)[1:][np.argmax(scores)]

0.01
[ 0.61616162  0.61616162  0.61616162] 0.616161616162
0.02
[ 0.61616162  0.61616162  0.61616162] 0.616161616162
0.03
[ 0.61616162  0.61279461  0.61616162] 0.615039281706
0.04
[ 0.61616162  0.61616162  0.61616162] 0.616161616162
0.05
[ 0.61952862  0.61616162  0.61616162] 0.617283950617
0.06
[ 0.61952862  0.61616162  0.61616162] 0.617283950617
0.07
[ 0.61952862  0.62289562  0.61616162] 0.619528619529
0.08
[ 0.62289562  0.63636364  0.61616162] 0.625140291807
0.09
[ 0.62626263  0.64309764  0.61616162] 0.628507295174
0.1
[ 0.63299663  0.65319865  0.61616162] 0.634118967452
0.11
[ 0.64983165  0.65656566  0.61616162] 0.640852974186
0.12
[ 0.64983165  0.66666667  0.62289562] 0.646464646465
0.13
[ 0.64983165  0.67676768  0.63636364] 0.654320987654
0.14
[ 0.65993266  0.67340067  0.64309764] 0.658810325477
0.15
[ 0.67340067  0.67340067  0.65319865] 0.666666666667
0.16
[ 0.67676768  0.67340067  0.66329966] 0.671156004489
0.17
[ 0.68686869  0.67676768  0.66329966] 0.675645342312
0.18
[ 0.690235

In [22]:
scores = []
for weight in np.arange(1, 100, 1)[1:]:
    print weight
    ret1, ret2 = test_m(LogisticRegression(C=0.23, class_weight={0.:1, 1.:weight}))
    scores.append(ret2)
print np.arange(1, 100, 1)[1:][np.argmax(scores)]

2
[ 0.75084175  0.77104377  0.79461279] 0.772166105499
3
[ 0.71717172  0.78114478  0.77777778] 0.758698092031
4
[ 0.68350168  0.74410774  0.75420875] 0.727272727273
5
[ 0.65993266  0.68686869  0.69360269] 0.680134680135
6
[ 0.64983165  0.61952862  0.59259259] 0.620650953984
7
[ 0.5959596   0.55555556  0.51515152] 0.555555555556
8
[ 0.57239057  0.48484848  0.47811448] 0.511784511785
9
[ 0.51851852  0.45791246  0.45454545] 0.476992143659
10
[ 0.5016835   0.44107744  0.42760943] 0.456790123457
11
[ 0.47811448  0.42087542  0.41414141] 0.43771043771
12
[ 0.45791246  0.4040404   0.4040404 ] 0.421997755331
13
[ 0.45791246  0.40740741  0.4006734 ] 0.421997755331
14
[ 0.44781145  0.41077441  0.3973064 ] 0.418630751964
15
[ 0.44107744  0.4040404   0.3973064 ] 0.414141414141
16
[ 0.43097643  0.4040404   0.3973064 ] 0.410774410774
17
[ 0.42760943  0.39393939  0.3973064 ] 0.406285072952
18
[ 0.42424242  0.39393939  0.39393939] 0.40404040404
19
[ 0.41750842  0.39057239  0.39393939] 0.400673400673
20

In [23]:
test_m(LogisticRegression())
test_m(LogisticRegression(C=0.23))
test_m(LogisticRegression(C=0.23, class_weight={0.:1, 1.:0.77}))

[ 0.78451178  0.78787879  0.78787879] 0.786756453423
[ 0.78787879  0.8013468   0.79124579] 0.793490460157
[ 0.77777778  0.81481481  0.81144781] 0.801346801347


(array([ 0.77777778,  0.81481481,  0.81144781]), 0.80134680134680136)

In [24]:
alg = LogisticRegression(C=0.23, class_weight={0.:1, 1.:0.77})
alg.fit(X, y)
pred = alg.predict(X_test)

In [25]:
print pred

[ 0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.
  0.  1.  0.  0.  1.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  1.  1.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.  1.  1.  0.  0.  1.  1.  0.
  1.  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.
  1.  0.  1.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  1.  1.  1.  1.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.  0.
  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.
  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.
  0.  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  1

In [26]:
ret = np.array([Passenger_Id.astype(int), pred.astype(int)]).transpose()
pd.DataFrame(ret, columns=['PassengerId', 'Survived']).to_csv('predictions.csv', index=False)